In [1]:
import os, sys
from pathlib import Path
src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

src_path

PosixPath('/Users/seobi/PythonProjects/AxDeepScholar/src')

In [2]:
from dotenv import load_dotenv
load_dotenv()


python-dotenv could not parse statement starting at line 2


True

In [3]:
from utils.custom_message_text import show_prompt
from core.prompts.PromptManager import PromptManager
from core.prompts.PromptType import PromptType
prompt_manager = PromptManager()
show_prompt(prompt_manager.get_prompt(PromptType.SYSTEM_CHECK_USER_NEED, messages=[], date=""))

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  오늘 날짜:                                                                                                     │
│  <사용자가 리포트 작성을 요청했었던 대화내역>                                                                   │
│  []                                                                                                             │
│  </사용자가 리포트 작성을 요청했었던 대화내역>                                                                  │
│                                                                                                                 │
│  이제 사용자가 제공한 정보만으로 AI 논문 리서치를 시작할 수 있는지,                                             │
│  아니면 추가로 사용자에게 질문을 해야 하는지 판단하십시오.                                                      │
│                                                                                                                 │
│  중요: 만약 이미 사용자에게 질문을 한 적이 없다면,                                                              │
│  다시 질문할 필요가 거의 없습니다. 정말 필요할 때만 새로 질문을 하십시오.                                       │
│                                                                                                                 │
│  약어나 모호한 용어, 의미 불분명한 표현이 있다면 사용자가 명확히 해주도록 요청하십시오                          │
│                                                                                                                 │
│  질문을 필요한 경우 다음 가이드를 따르세요:                                                                     │
│  - 불필요한 장황함 없이 필요한 정보만 간결히 물어보세요.                                                        │
│  - 리서치 수행에 필꼭 필요한 모든 정보를 잘 구조화해 요청하세요.                                                │
│  - 필요하다면 마크다운 서식이 적용되는 **리스트나 번호 목록**으로 작성하세요.                                   │
│  - 이미 사용자가 제공한 정보를 반복해서 묻지마세요.                                                             │
│                                                                                                                 │
│  다음과 같은 JSON 형식으로 응답해야 합니다:                                                                     │
│  "need_add_question": boolean,                                                                                  │
│  "question": "<사용자에게 추가 질문>",                                                                          │
│  "start_research_message": "<충분한 정보가 있을 때 리서치 시작을 알리는 메시지>"                                │
│                                                                                                                 │
│  추가 사용자 질문이 필요한 경우:                                                                                │
│  "need_add_question": true,                                                                                     │
│  "question": "<당신의 질문>"                                                                                    │
│  "start_research_message": ""                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
# %%writefile main_agent.py

from datetime import datetime
from langchain.chat_models import init_chat_model
from main_state import MainInputState, MainState, ClarifyWithUser, ResearchQuestion
from langgraph.graph.state import Command, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from utils.helper import get_today_str
from core.prompts.PromptManager import PromptManager
from core.prompts.PromptType import PromptType

from agent.research.research_agent import graph as research_graph
from agent.supervisor.supervisor_agent import graph as supervisor_graph

prompt_manager = PromptManager()
model = init_chat_model(model="gpt-4.1", temperature=0)
final_model = init_chat_model(model="gpt-5.1", model_provider="openai")


def clarify_with_user(
    state: MainState,
) -> Command[Literal["write_research_brief", "__end__"]]:
    clarify_prompt = prompt_manager.get_prompt(
        PromptType.SYSTEM_CHECK_USER_NEED,
        messages=state["messages"],
        date=get_today_str(),
    )
    parser_model = model.with_structured_output(ClarifyWithUser)
    response: ClarifyWithUser = parser_model.invoke(
        [HumanMessage(content=clarify_prompt)]
    )

    if response.need_clarification:
        return Command(
            goto=END, update={"messages": [AIMessage(content=response.question)]}
        )
    else:
        return Command(
            goto="write_research_brief",
            update={"messages": [AIMessage(content=response.verification)]},
        )


def write_research_brief(state: MainState) -> MainState:
    parser_model = model.with_structured_output(ResearchQuestion)
    start_prompt = prompt_manager.get_prompt(
        PromptType.START_RESEARCH, messages=state["messages"], date=get_today_str()
    )
    response: ResearchQuestion = parser_model.invoke(
        [HumanMessage(content=start_prompt)]
    )
    return {
        "research_brief": response.research_brief,
        "supervisor_messages": [HumanMessage(content=response.research_brief)],
    }


async def final_report(state: MainState) -> MainState:
    notes = state.get("notes", [])
    findings = "\n".join(notes)
    final_report_prompt = prompt_manager.get_prompt(
        PromptType.FINAL_PEPORT,
        research_brief=state.get("research_brief", ""),
        date=get_today_str(),
        findings=findings,
    )
    final_report = await final_model.ainvoke(
        [HumanMessage(content=final_report_prompt)]
    )
    return {
        "final_report": final_report.content,
        "messages": ["이것이 최종 리포트 입니다." + final_report.content],
    }


graph_builder = StateGraph(MainState, input_schema=MainInputState)
graph_builder.add_node("clarify_with_user", clarify_with_user)
graph_builder.add_node("write_research_brief", write_research_brief)
graph_builder.add_node("supervisor_graph",supervisor_graph)
graph_builder.add_node("final_report",final_report)


graph_builder.add_edge(START, "clarify_with_user")
graph_builder.add_edge("write_research_brief", "supervisor_graph")
graph_builder.add_edge("supervisor_graph", "final_report")
graph_builder.add_edge("final_report",END)

graph = graph_builder.compile()

/Users/seobi/PythonProjects/AxDeepScholar/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/AxDeepScholar/src/core/rag/AxrivRetriever.py:40: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


기존 DB 불러오기 실패 'NoneType' object is not iterable


/Users/seobi/PythonProjects/AxDeepScholar/src/core/rag/AxrivRetriever.py:111: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.db.persist()


DB 생성 및 저장: /Users/seobi/PythonProjects/AxDeepScholar/.chroma_db


In [7]:
from agent.main.main_agent import graph_builder as main_builder
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()
graph = main_builder.compile(checkpointer=checkpointer)


In [ ]:
from langchain_core.messages import HumanMessage

thread = {"configurable": {"thread_id": "1", "recursion_limit": 50}}
result = await graph.ainvoke(
    {
        "messages": [
            HumanMessage(
                content="""
AI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.
기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 
도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         
"""
            )
        ]
    },
    config=thread,
)

print(result["messages"])

[HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'), AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460')]


[HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'),
 AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460')]

In [ ]:
result = await graph.ainvoke(
    {
        "messages": result["messages"] + [HumanMessage(content="""산업율이라는 말은 돈이 된다는 뜻입니다. 
                                                       기존 에이전트 기술이 실제 산업군에 얼마나 도움이 될 수 있는지 알고 싶고, 그럴만한 특별한 기술이 있을지 궁금합니다.
                                                       기존 논문 내용이나 지금까지 나온 기술중에서 그러한 힌트를 참고하고 싶습니다.""")]
    },
    config=thread,
)
print(result["messages"])

[HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'), AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460'), HumanMessage(content='산업율이라는 말은 돈이 된다는 뜻입니다. 기존 에이전트 기술이 실제 산업군에 얼마나 도움이 될 수 있는지 알고 싶고, 그럴만한 특별한 기술이 있을지 궁금합니다. 기존 논문 내용이나 지금까지 나온 기술중에서 그러한 힌트를 참고하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='9737d301-7b89-46fd-9131-3b163d7f4f5d'), AIMessage(content="AI 에이전트의 실제 산업 적용 가능성과 경제적 효과(돈이 되는 기술)에 대한 논문 리서치를 요청하셨습니다. 추가로 다음 두 가지를 명확히 해주시면 더 구체적이고 실질적인 리서치가 가능합니다:\n\n1. 관심 있는 산업군(예: 제조, 금융, 서비스, 물류 등)이 있

In [11]:
result["messages"]

[HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'),
 AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460'),
 HumanMessage(content='산업율이라는 말은 돈이 된다는 뜻입니다. 기존 에이전트 기술이 실제 산업군에 얼마나 도움이 될 수 있는지 알고 싶고, 그럴만한 특별한 기술이 있을지 궁금합니다. 기존 논문 내용이나 지금까지 나온 기술중에서 그러한 힌트를 참고하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='9737d301-7b89-46fd-9131-3b163d7f4f5d'),
 AIMessage(content="AI 에이전트의 실제 산업 적용 가능성과 경제적 효과(돈이 되는 기술)에 대한 논문 리서치를 요청하셨습니다. 추가로 다음 두 가지를 명확히 해주시면 더 구체적이고 실질적인 리서치가 가능합니다:\n\n1. 관심 있는 산업군(예: 제조, 금융, 서비스, 물류 등)

In [ ]:

result = await graph.ainvoke(
    {
        "messages":result["messages"] + [HumanMessage(content="1. 포괄적으로 다루고 싶습니다. 2. 에이전트 전반입니다.")]    
    },
    config=thread,
)
result

Error in supervisor tools: 'messages'


[HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'),
 AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460'),
 HumanMessage(content='산업율이라는 말은 돈이 된다는 뜻입니다. 기존 에이전트 기술이 실제 산업군에 얼마나 도움이 될 수 있는지 알고 싶고, 그럴만한 특별한 기술이 있을지 궁금합니다. 기존 논문 내용이나 지금까지 나온 기술중에서 그러한 힌트를 참고하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='9737d301-7b89-46fd-9131-3b163d7f4f5d'),
 AIMessage(content="AI 에이전트의 실제 산업 적용 가능성과 경제적 효과(돈이 되는 기술)에 대한 논문 리서치를 요청하셨습니다. 추가로 다음 두 가지를 명확히 해주시면 더 구체적이고 실질적인 리서치가 가능합니다:\n\n1. 관심 있는 산업군(예: 제조, 금융, 서비스, 물류 등)

In [17]:
print(result["messages"][-1].content)

이것이 최종 리포트 입니다.# 산업 전반에서 AI 에이전트의 경제적 가치 극대화: 메커니즘, 사례, 성공 요인 종합 보고서

## 1. 개요

AI 에이전트(LLM 기반 자동화/의사결정 시스템)는 이미 제조, 금융, 서비스, 물류 등 주요 산업에서 **비용 절감·생산성 향상·매출 증대**를 실질적으로 만들어내고 있다.  
이 보고서는 다음을 중심으로 정리한다.

1. 산업별 적용 사례와 **정량적 성과**
2. 경제적 가치를 만들어내는 **구체적 메커니즘**
3. 토큰 비용·인프라 등 **비용 구조와 ROI**
4. 최신 연구·제품 동향과 **성공 요인, 실무 적용 가이드**

---

## 2. AI 에이전트의 정의와 범위

### 2.1 정의

이 보고서에서의 “AI 에이전트”는 다음 특성을 가진 시스템을 의미한다.

- **LLM 또는 ML 모델을 기반**으로
- **목표(goal)를 부여받고**, 이를 위해
- **외부 도구(데이터베이스, API, 소프트웨어)**를 호출하거나 작업을 스스로 계획·수행하며
- 필요 시 **인간에게 질의/승인 요청(Human-in-the-loop)**을 하고
- **연속적인 상태(state)를 인지**하며, 여러 스텝에 걸친 의사결정을 수행하는 시스템

즉, 단순 질의응답형 챗봇을 넘어,  
“업무 프로세스의 일부 또는 전부를 맡아 수행하는 자율/반자율 소프트웨어 노동자”에 가까운 개념이다.

### 2.2 범위

- 산업: 제조, 금융, 서비스(콜센터·보험·리테일·헬스케어 등), 물류/공급망, IT/소프트웨어, 공공 등 전 산업
- 기술: LLM 에이전트, 멀티에이전트, RAG+에이전트, 툴 사용/플래너·익스퍼트 구조, 온디바이스·경량 모델
- 가치 지표: 비용 절감, 생산성 향상, 리드타임 단축, 품질 개선, 매출/전환율 증대, 규제·리스크 관리 등

---

## 3. 산업별 적용 사례와 정량적 성과

### 3.1 제조: 설계·품질·생산계획 자동화

#### 3.1.1 공정 최적화·품질 관리

- **Seagate 

In [18]:
result

{'messages': [HumanMessage(content='\nAI 에이전트 개발 중의 99 프로에 가까운 산업율을 만들 수 있는 최고의 연구가 필요합니다.\n기존 에이전트 개발은 기능 대비 토큰 비용이 많이 나갑니다. 토큰 비용을 계산해서 실제 산업에서 \n도움되고 강력한 기술을 보여줄 수 있는 논문을 써주세요.                                         \n', additional_kwargs={}, response_metadata={}, id='d700ba09-d1b7-43e7-b080-208311f1cb50'),
  AIMessage(content="AI 에이전트 개발에서 '99%에 가까운 산업율'이 의미하는 바를 구체적으로 설명해주실 수 있나요? 예를 들어, 산업 적용률, 자동화율, 효율성 등 어떤 지표를 의미하는지 명확히 해주시면 논문 방향을 더 정확히 잡을 수 있습니다. 또한, 적용하고자 하는 산업 분야(예: 제조, 금융, 서비스 등)가 있다면 알려주세요.", additional_kwargs={}, response_metadata={}, id='cb691a56-2630-4f0c-9217-09e49589e460'),
  HumanMessage(content='산업율이라는 말은 돈이 된다는 뜻입니다. 기존 에이전트 기술이 실제 산업군에 얼마나 도움이 될 수 있는지 알고 싶고, 그럴만한 특별한 기술이 있을지 궁금합니다. 기존 논문 내용이나 지금까지 나온 기술중에서 그러한 힌트를 참고하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='9737d301-7b89-46fd-9131-3b163d7f4f5d'),
  AIMessage(content="AI 에이전트의 실제 산업 적용 가능성과 경제적 효과(돈이 되는 기술)에 대한 논문 리서치를 요청하셨습니다. 추가로 다음 두 가지를 명확히 해주시면 더 구체적이고 실질적인 리서치가 가능합니다:\n\n1. 관심 있는 산업군(예: 제조